#Install needed libraries

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/"
%cd {path}
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1


In [ ]:
!git clone https://github.com/phamdinhkhanh/vnquant
%cd vnquant
!python setup.py install

Cloning into 'vnquant'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 341 (delta 65), reused 89 (delta 53), pack-reused 237
Receiving objects: 100% (341/341), 1.05 MiB | 6.61 MiB/s, done.
Resolving deltas: 100% (164/164), done.
/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/vnquant
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/

In [ ]:
%cd ..
!rm -rf vnquant
!ls

/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1


In [ ]:
!pip freeze | grep vnquant

vnquant==0.1.1


In [ ]:
import vnquant
vnquant.__version__

'0.1.1'

In [ ]:
import vnquant.data as dt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import accuracy_score, classification_report

#We choose 10 codes from biggest companies on stock exchange in Vietnam

In [ ]:
stock_symbols = ['VIC', 'VCB', 'VNM', 'GAS', 'HVN', 'MWG', 'FPT', 'HPG', 'MSN', 'SAB']

#Prepare the dataset

In [ ]:
stock_data = {}

for symbol in stock_symbols:
  df = dt.DataLoader(symbols=symbol,
                     start="2014-01-01",
                     end="2024-01-01",
                     minimal=True,
                     data_source="VND")
  stock_data[symbol] = df.download()

In [ ]:
stock_data[stock_symbols[0]]

Attributes,high,low,open,close,avg,volume
Symbols,VIC,VIC,VIC,VIC,VIC,VIC
date,,,,,,
2014-01-02,70.50,69.50,70.00,70.00,69.92,74950.0
2014-01-03,70.50,69.50,70.00,70.50,70.00,91920.0
2014-01-06,70.50,69.50,70.50,70.50,70.11,111760.0
2014-01-07,70.50,70.00,70.50,70.50,70.47,294120.0
2014-01-08,70.50,70.00,70.00,70.00,70.02,109890.0
...,...,...,...,...,...,...
2023-12-25,43.55,43.00,43.10,43.40,43.34,3364500.0
2023-12-26,43.75,43.35,43.40,43.55,43.55,1806700.0


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]] = stock_data[stock_symbols[i]].ewm(alpha=0.65).mean()

stock_data[stock_symbols[0]].head()

Attributes,high,low,open,close,avg,volume
Symbols,VIC,VIC,VIC,VIC,VIC,VIC
date,,,,,,
2014-01-02,70.5,69.500000,70.000000,70.000000,69.920000,74950.000000
2014-01-03,70.5,69.500000,70.000000,70.370370,69.979259,87520.370370
2014-01-06,70.5,69.500000,70.339559,70.458404,70.068048,103981.918506
2014-01-07,70.5,69.829951,70.445434,70.485853,70.333297,229454.545080
2014-01-08,70.5,69.941067,70.154373,70.168381,70.128579,151327.251584


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['today', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['close'].pct_change() * 100

stock_data[stock_symbols[1]].head()

Attributes,high,low,open,close,avg,volume,today
Symbols,VCB,VCB,VCB,VCB,VCB,VCB,VCB
date,,,,,,,
2014-01-02,26.900000,26.600000,26.900000,26.700000,26.740000,3.339000e+05,NaN
2014-01-03,27.196296,26.674074,26.751852,27.070370,26.984444,6.890556e+05,1.387155
2014-01-06,27.334635,26.895416,27.124109,27.090492,27.110458,4.763795e+05,0.074332
2014-01-07,27.707721,27.228392,27.372160,27.492708,27.486300,1.727654e+06,1.484712
2014-01-08,27.964049,27.471213,27.651724,27.889532,27.730488,1.562150e+06,1.443379


In [ ]:
for i in range(len(stock_symbols)):
  for j in range(1, 6):
    stock_data[stock_symbols[i]][f'previous{str(j)}', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['today'].shift(j)

stock_data[stock_symbols[2]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5
Symbols,VNM,VNM,VNM,VNM,VNM,VNM,VNM,VNM,VNM,VNM,VNM,VNM
date,,,,,,,,,,,,
2014-01-02,137.000000,135.0,135.000000,135.000000,135.550000,32520.000000,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,136.259259,135.0,135.000000,135.740741,135.527778,31171.851852,0.548697,NaN,NaN,NaN,NaN,NaN
2014-01-06,136.083192,135.0,135.000000,135.916808,135.610781,62355.653650,0.129709,0.548697,NaN,NaN,NaN,NaN
2014-01-07,136.028293,135.0,135.659903,135.311804,135.412295,91070.886744,-0.445128,0.129709,0.548697,NaN,NaN,NaN
2014-01-08,136.009806,135.0,135.228701,135.761493,135.534947,56980.762637,0.332336,-0.445128,0.129709,0.548697,NaN,NaN


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['ema50', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['close'] / stock_data[stock_symbols[i]]['close'].ewm(50).mean()
  stock_data[stock_symbols[i]]['ema21', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['close'] / stock_data[stock_symbols[i]]['close'].ewm(21).mean()
  stock_data[stock_symbols[i]]['ema14', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['close'] / stock_data[stock_symbols[i]]['close'].ewm(14).mean()
  stock_data[stock_symbols[i]]['ema5', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['close'] / stock_data[stock_symbols[i]]['close'].ewm(5).mean()

stock_data[stock_symbols[3]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5,ema50,ema21,ema14,ema5
Symbols,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS,GAS
date,,,,,,,,,,,,,,,,
2014-01-02,66.500000,65.500000,66.500000,66.500000,66.080000,229950.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
2014-01-03,66.500000,65.500000,66.500000,66.129630,65.976296,249964.814815,-0.556948,NaN,NaN,NaN,NaN,NaN,0.997235,0.997272,0.997304,0.997461
2014-01-06,66.839559,65.839559,66.160441,66.720713,66.270832,200155.093379,0.893825,-0.556948,NaN,NaN,NaN,NaN,1.004050,1.004019,1.003992,1.003844
2014-01-07,67.275386,66.275386,66.714468,66.905015,66.705819,169367.272952,0.276229,0.893825,-0.556948,NaN,NaN,NaN,1.005058,1.004965,1.004886,1.004477
2014-01-08,67.422156,66.422156,67.227760,67.293797,67.061404,177190.635299,0.581096,0.276229,0.893825,-0.556948,NaN,NaN,1.008612,1.008418,1.008254,1.007419


In [ ]:
def rsi(X, window=14):
  delta = X.diff(1)

  gains = delta.where(delta > 0, 0)
  losses = -delta.where(delta < 0, 0)

  avg_gains = gains.rolling(window=window, min_periods=1).mean()
  avg_losses = losses.rolling(window=window, min_periods=1).mean()

  rs = avg_gains / avg_losses
  rsi = 100 - (100 / (1 + rs))

  return rsi

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['rsi', str(stock_symbols[i])] = rsi(stock_data[stock_symbols[i]]['close'])

stock_data[stock_symbols[4]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5,ema50,ema21,ema14,ema5,rsi
Symbols,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN,HVN
date,,,,,,,,,,,,,,,,,
2017-01-03,39.200000,39.200000,39.200000,39.200000,39.200000,700.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,NaN
2017-01-04,43.496296,43.496296,43.496296,43.496296,43.496296,6750.370370,10.959940,NaN,NaN,NaN,NaN,NaN,1.051412,1.050683,1.050072,1.047008,100.000000
2017-01-05,49.067572,45.196604,49.067572,47.437691,48.052971,521133.820034,9.061449,10.959940,NaN,NaN,NaN,NaN,1.092220,1.090380,1.088840,1.081195,100.000000
2017-01-06,51.134670,44.406962,51.134670,45.169150,46.837449,342822.959663,-4.782150,9.061449,10.959940,NaN,NaN,NaN,1.029390,1.027717,1.026338,1.019810,78.407660
2017-01-09,47.910199,42.180744,47.910199,42.967640,43.843784,223068.513075,-4.873924,-4.782150,9.061449,10.95994,NaN,NaN,0.983469,0.982441,0.981627,0.978254,64.824189


In [ ]:
def macd(X, short_window=12, long_window=29, signal=9):
  short_ema = X.ewm(span=short_window, adjust=False).mean()

  long_ema = X.ewm(span=long_window, adjust=False).mean()

  macd = short_ema - long_ema

  signal = macd.ewm(span=signal, adjust=False).mean()

  return signal

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['macd', str(stock_symbols[i])] = macd(stock_data[stock_symbols[i]]['close'])

stock_data[stock_symbols[5]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5,ema50,ema21,ema14,ema5,rsi,macd
Symbols,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG
date,,,,,,,,,,,,,,,,,,
2014-07-14,81.500000,81.000000,81.000000,81.500000,81.450000,10660.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,NaN,0.000000
2014-07-15,85.574074,85.444444,85.444444,85.574074,85.561111,12748.888889,4.998864,NaN,NaN,NaN,NaN,NaN,1.024138,1.023804,1.023524,1.022119,100.0,0.071035
2014-07-16,90.617148,90.575552,90.575552,90.617148,90.612988,15832.835314,5.893226,4.998864,NaN,NaN,NaN,NaN,1.054212,1.053216,1.052380,1.048204,100.0,0.271165
2014-07-17,96.478966,96.464819,96.464819,96.478966,96.477551,68154.646540,6.468773,5.893226,4.998864,NaN,NaN,NaN,1.088113,1.086065,1.084349,1.075834,100.0,0.644448
2014-07-18,102.700314,102.695411,102.695411,102.700314,102.699823,111480.682755,6.448398,6.468773,5.893226,4.998864,NaN,NaN,1.121361,1.117879,1.114968,1.100672,100.0,1.215521


In [ ]:
window = 6
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['roc', str(stock_symbols[i])] = ((stock_data[stock_symbols[i]]['close'] - stock_data[stock_symbols[i]]['close'].shift(window)) / stock_data[stock_symbols[i]]['close'].shift(window)) * 100

stock_data[stock_symbols[6]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5,ema50,ema21,ema14,ema5,rsi,macd,roc
Symbols,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT
date,,,,,,,,,,,,,,,,,,,
2014-01-02,47.800000,47.200000,47.600000,47.400000,47.550000,207470.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,NaN
2014-01-03,47.503704,47.051852,47.377778,47.325926,47.305556,176092.222222,-0.156274,NaN,NaN,NaN,NaN,NaN,0.999226,0.999236,0.999245,0.999289,0.000000,-0.001292,NaN
2014-01-06,47.840747,47.016638,47.324958,47.783701,47.342530,261374.244482,0.967282,-0.156274,NaN,NaN,NaN,NaN,1.005851,1.005777,1.005715,1.005393,86.072358,0.004650,NaN
2014-01-07,47.945838,47.335610,47.770420,47.596486,47.637817,280917.760455,-0.391797,0.967282,-0.156274,NaN,NaN,NaN,1.001417,1.001344,1.001283,1.000986,63.662614,0.011589,NaN
2014-01-08,47.981229,47.377684,47.528376,47.729469,47.743792,182041.901609,0.279396,-0.391797,0.967282,-0.156274,NaN,NaN,1.003336,1.003232,1.003146,1.002725,69.333942,0.021105,NaN


In [ ]:
window = 16
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['high-low', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['high', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['low', str(stock_symbols[i])]
  stock_data[stock_symbols[i]]['high-preclose', str(stock_symbols[i])] = abs(stock_data[stock_symbols[i]]['high', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['close', str(stock_symbols[i])].shift(1))
  stock_data[stock_symbols[i]]['low-preclose', str(stock_symbols[i])] = abs(stock_data[stock_symbols[i]]['low', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['close', str(stock_symbols[i])].shift(1))

  stock_data[stock_symbols[i]]['tr', str(stock_symbols[i])] = stock_data[stock_symbols[i]][[('high-low', str(stock_symbols[i])), ('high-preclose', str(stock_symbols[i])), ('low-preclose', str(stock_symbols[i]))]].max(axis=1)

  stock_data[stock_symbols[i]]['atr', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['tr', str(stock_symbols[i])].rolling(window=window).mean()

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]] = stock_data[stock_symbols[i]].drop([('high-low', str(stock_symbols[i])), ('high-preclose', str(stock_symbols[i])), ('low-preclose', str(stock_symbols[i])), ('tr', str(stock_symbols[i]))], axis=1)

stock_data[stock_symbols[7]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,previous4,previous5,ema50,ema21,ema14,ema5,rsi,macd,roc,atr
Symbols,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG
date,,,,,,,,,,,,,,,,,,,,
2014-01-02,41.500000,41.000000,41.500000,41.100000,41.130000,276250.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,NaN,NaN
2014-01-03,41.574074,41.000000,41.277778,41.470370,41.300370,424864.814815,0.901144,NaN,NaN,NaN,NaN,NaN,1.004441,1.004381,1.004330,1.004076,100.0,0.006458,NaN,NaN
2014-01-06,42.134975,41.339559,41.768251,42.033786,41.802666,443856.451613,1.358598,0.901144,NaN,NaN,NaN,NaN,1.011865,1.011663,1.011492,1.010634,100.0,0.026481,NaN,NaN
2014-01-07,42.573827,41.907366,42.251134,42.143471,42.302433,337198.725563,0.260945,1.358598,0.901144,NaN,NaN,NaN,1.010731,1.010434,1.010185,1.008961,100.0,0.055928,NaN,NaN
2014-01-08,42.590929,41.967896,42.087035,42.376438,42.418461,350568.775959,0.552796,0.260945,1.358598,0.901144,NaN,NaN,1.012880,1.012467,1.012122,1.010445,100.0,0.093794,NaN,NaN


In [ ]:
def obv(X):
  obv = pd.Series(index=X.index)
  obv.iloc[0] = 0

  for i in range(1, len(X)):
    if (X['close'].iloc[i].values > X['close'].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] + X['volume'].iloc[i].values
    elif (X['close'].iloc[i].values < X['close'].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] - X['volume'].iloc[i].values
    else:
      obv.iloc[i] = obv.iloc[i-1]

  return obv

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['obv', str(stock_symbols[i])] = obv(stock_data[stock_symbols[i]])

stock_data[stock_symbols[6]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,previous5,ema50,ema21,ema14,ema5,rsi,macd,roc,atr,obv
Symbols,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,...,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,47.800000,47.200000,47.600000,47.400000,47.550000,207470.000000,NaN,NaN,NaN,NaN,...,NaN,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,NaN,NaN,0.000000
2014-01-03,47.503704,47.051852,47.377778,47.325926,47.305556,176092.222222,-0.156274,NaN,NaN,NaN,...,NaN,0.999226,0.999236,0.999245,0.999289,0.000000,-0.001292,NaN,NaN,-176092.222222
2014-01-06,47.840747,47.016638,47.324958,47.783701,47.342530,261374.244482,0.967282,-0.156274,NaN,NaN,...,NaN,1.005851,1.005777,1.005715,1.005393,86.072358,0.004650,NaN,NaN,85282.022260
2014-01-07,47.945838,47.335610,47.770420,47.596486,47.637817,280917.760455,-0.391797,0.967282,-0.156274,NaN,...,NaN,1.001417,1.001344,1.001283,1.000986,63.662614,0.011589,NaN,NaN,-195635.738195
2014-01-08,47.981229,47.377684,47.528376,47.729469,47.743792,182041.901609,0.279396,-0.391797,0.967282,-0.156274,...,NaN,1.003336,1.003232,1.003146,1.002725,69.333942,0.021105,NaN,NaN,-13593.836586


In [ ]:
def emv(X):
  emv = pd.Series(index=X.index)
  emv.iloc[0] = np.nan

  for i in range(1, len(X)):
    dm = 0.5 * ((X['high'].iloc[i].values + X['low'].iloc[i].values) - (X['high'].iloc[i-1].values + X['low'].iloc[i-1].values))
    br = X['volume'].iloc[i].values / (1000000 * (X['high'].iloc[i].values - X['low'].iloc[i].values))
    emv.iloc[i] = dm / br if br != 0 else 0

  return emv

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['emv', str(stock_symbols[i])] = emv(stock_data[stock_symbols[i]])

<ipython-input-45-fcf8d000722f>:7: RuntimeWarning: divide by zero encountered in divide
  br = X['volume'].iloc[i].values / (1000000 * (X['high'].iloc[i].values - X['low'].iloc[i].values))
<ipython-input-45-fcf8d000722f>:7: RuntimeWarning: divide by zero encountered in divide
  br = X['volume'].iloc[i].values / (1000000 * (X['high'].iloc[i].values - X['low'].iloc[i].values))


In [ ]:
stock_data[stock_symbols[7]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,ema50,ema21,ema14,ema5,rsi,macd,roc,atr,obv,emv
Symbols,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,...,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,41.500000,41.000000,41.500000,41.100000,41.130000,276250.000000,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,NaN,NaN,0.000000e+00,NaN
2014-01-03,41.574074,41.000000,41.277778,41.470370,41.300370,424864.814815,0.901144,NaN,NaN,NaN,...,1.004441,1.004381,1.004330,1.004076,100.0,0.006458,NaN,NaN,4.248648e+05,0.050044
2014-01-06,42.134975,41.339559,41.768251,42.033786,41.802666,443856.451613,1.358598,0.901144,NaN,NaN,...,1.011865,1.011663,1.011492,1.010634,100.0,0.026481,NaN,NaN,8.687213e+05,0.806837
2014-01-07,42.573827,41.907366,42.251134,42.143471,42.302433,337198.725563,0.260945,1.358598,0.901144,NaN,...,1.010731,1.010434,1.010185,1.008961,100.0,0.055928,NaN,NaN,1.205920e+06,0.994812
2014-01-08,42.590929,41.967896,42.087035,42.376438,42.418461,350568.775959,0.552796,0.260945,1.358598,0.901144,...,1.012880,1.012467,1.012122,1.010445,100.0,0.093794,NaN,NaN,1.556489e+06,0.068984


In [ ]:
def mfi(X, window=14):
  combine_price = (X['high'] + X['low'] + X['close']) / 3

  raw_money_flow = combine_price * X['volume']

  flow_direction = (combine_price.diff() > 0).astype(int)

  positive_money_flow = flow_direction * raw_money_flow
  negative_money_flow = (1 - flow_direction) * raw_money_flow

  positive = positive_money_flow.rolling(window=window, min_periods=1).sum()
  negative = negative_money_flow.rolling(window=window, min_periods=1).sum()

  mf = positive / negative
  mfi = 100 - (100 / (1 + mf))

  return mfi

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['mfi', str(stock_symbols[i])] = mfi(stock_data[stock_symbols[i]])

stock_data[stock_symbols[8]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,ema21,ema14,ema5,rsi,macd,roc,atr,obv,emv,mfi
Symbols,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,...,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,83.000000,81.500000,83.000000,82.000000,82.240000,92560.000000,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000
2014-01-03,82.629630,81.500000,82.259259,81.629630,81.884444,59922.962963,-0.451671,NaN,NaN,NaN,...,0.997789,0.997814,0.997942,0.000000,-0.006458,NaN,NaN,-59922.962963,-3.490993,0.000000
2014-01-06,82.541596,81.500000,82.083192,81.881154,81.942547,100389.541596,0.308129,-0.451671,NaN,NaN,...,1.000562,1.000572,1.000614,40.444894,-0.012272,NaN,NaN,40466.578633,-0.456699,39.671831
2014-01-07,83.174049,81.829951,82.028293,82.289532,82.409397,114122.418543,0.498745,0.308129,-0.451671,NaN,...,1.004060,1.004018,1.003794,64.051241,-0.010242,NaN,NaN,154588.997176,5.667450,58.494597
2014-01-08,84.040468,82.267783,82.989953,83.733923,83.115498,139748.439155,1.755254,0.498745,0.308129,-0.451671,...,1.016854,1.016614,1.015355,85.033504,0.021847,NaN,NaN,294337.436332,8.272094,70.060592


In [ ]:
def cmf(X, window=14):

  money_flow_multiplier = ((X['close'] - X['low']) - (X['high'] - X['close']))/(X['high']-X['low'])

  money_flow_volume = money_flow_multiplier * X['volume']

  cmf = money_flow_volume.rolling(window=window).sum() / X['volume'].rolling(window=window).sum()

  return cmf

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['cmf', str(stock_symbols[i])] = cmf(stock_data[stock_symbols[i]])

stock_data[stock_symbols[8]].tail()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,ema14,ema5,rsi,macd,roc,atr,obv,emv,mfi,cmf
Symbols,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,...,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN
date,,,,,,,,,,,,,,,,,,,,,
2023-12-25,65.699806,63.104347,63.356224,65.284186,64.733563,3.247911e+06,3.278220,-0.618872,0.276401,1.394464,...,1.019148,1.020954,64.621763,-0.315303,2.291279,1.723156,1.288123e+08,0.664751,61.762518,-0.118617
2023-12-26,66.479932,64.726521,65.659679,65.879465,65.530747,2.879549e+06,0.911828,3.278220,-0.618872,0.276401,...,1.026495,1.025093,65.324652,-0.242928,4.202320,1.741275,1.316918e+08,0.731403,61.519508,-0.109521
2023-12-27,67.402976,65.554283,66.010887,66.607813,66.518261,5.939977e+06,1.105576,0.911828,3.278220,-0.618872,...,1.035232,1.030172,64.500420,-0.144114,6.473818,1.784179,1.376318e+08,0.272451,62.992487,-0.109644
2023-12-28,67.531042,66.233999,66.718811,66.862734,66.922391,5.726792e+06,0.382720,1.105576,0.911828,3.278220,...,1.036485,1.028268,58.367050,-0.028821,5.411391,1.803258,1.433586e+08,0.091476,63.596304,-0.151003
2023-12-29,68.030865,66.731900,66.966584,66.951957,67.362837,1.174190e+07,0.133441,0.382720,1.105576,0.911828,...,1.035255,1.024579,51.648075,0.092704,5.261111,1.777435,1.551005e+08,0.055187,65.741145,-0.303429


In [ ]:
window = 16
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['minimum_low', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['low', str(stock_symbols[i])].rolling(window=window).min()
  stock_data[stock_symbols[i]]['maximum_high', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['high', str(stock_symbols[i])].rolling(window=window).max()

  stock_data[stock_symbols[i]]['stoch', str(stock_symbols[i])] = ((stock_data[stock_symbols[i]]['close', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['minimum_low', str(stock_symbols[i])]) / (stock_data[stock_symbols[i]]['maximum_high', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['minimum_low', str(stock_symbols[i])])) * 100

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]] = stock_data[stock_symbols[i]].drop([('minimum_low', str(stock_symbols[i])), ('maximum_high', str(stock_symbols[i]))], axis=1)

stock_data[stock_symbols[7]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,ema5,rsi,macd,roc,atr,obv,emv,mfi,cmf,stoch
Symbols,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,...,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,41.500000,41.000000,41.500000,41.100000,41.130000,276250.000000,NaN,NaN,NaN,NaN,...,1.000000,NaN,0.000000,NaN,NaN,0.000000e+00,NaN,0.000000,NaN,NaN
2014-01-03,41.574074,41.000000,41.277778,41.470370,41.300370,424864.814815,0.901144,NaN,NaN,NaN,...,1.004076,100.0,0.006458,NaN,NaN,4.248648e+05,0.050044,60.684135,NaN,NaN
2014-01-06,42.134975,41.339559,41.768251,42.033786,41.802666,443856.451613,1.358598,0.901144,NaN,NaN,...,1.010634,100.0,0.026481,NaN,NaN,8.687213e+05,0.806837,76.048074,NaN,NaN
2014-01-07,42.573827,41.907366,42.251134,42.143471,42.302433,337198.725563,0.260945,1.358598,0.901144,NaN,...,1.008961,100.0,0.055928,NaN,NaN,1.205920e+06,0.994812,81.568617,NaN,NaN
2014-01-08,42.590929,41.967896,42.087035,42.376438,42.418461,350568.775959,0.552796,0.260945,1.358598,0.901144,...,1.010445,100.0,0.093794,NaN,NaN,1.556489e+06,0.068984,85.138509,NaN,NaN


In [ ]:
_datawindow = 21
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['combine_price', str(stock_symbols[i])] = (stock_data[stock_symbols[i]]['high', str(stock_symbols[i])] + stock_data[stock_symbols[i]]['low', str(stock_symbols[i])] + stock_data[stock_symbols[i]]['close', str(stock_symbols[i])]) / 3

  stock_data[stock_symbols[i]]['sma_combine_price', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['combine_price', str(stock_symbols[i])].rolling(window=window).mean()
  stock_data[stock_symbols[i]]['mean_deviation', str(stock_symbols[i])] = stock_data[stock_symbols[i]]['combine_price', str(stock_symbols[i])].rolling(window).apply(lambda x: x.mad())

  stock_data[stock_symbols[i]]['cci', str(stock_symbols[i])] = (stock_data[stock_symbols[i]]['combine_price', str(stock_symbols[i])] - stock_data[stock_symbols[i]]['sma_combine_price', str(stock_symbols[i])]) / (0.015 * stock_data[stock_symbols[i]]['mean_deviation', str(stock_symbols[i])])

In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]] = stock_data[stock_symbols[i]].drop([('combine_price', str(stock_symbols[i])), ('sma_combine_price', str(stock_symbols[i])), ('mean_deviation', str(stock_symbols[i]))], axis=1)

stock_data[stock_symbols[5]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,rsi,macd,roc,atr,obv,emv,mfi,cmf,stoch,cci
Symbols,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,...,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG,MWG
date,,,,,,,,,,,,,,,,,,,,,
2014-07-14,81.500000,81.000000,81.000000,81.500000,81.450000,10660.000000,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
2014-07-15,85.574074,85.444444,85.444444,85.574074,85.561111,12748.888889,4.998864,NaN,NaN,NaN,...,100.0,0.071035,NaN,NaN,12748.888889,43.307790,55.706701,NaN,NaN,NaN
2014-07-16,90.617148,90.575552,90.575552,90.617148,90.612988,15832.835314,5.893226,4.998864,NaN,NaN,...,100.0,0.271165,NaN,NaN,28581.724203,13.364772,74.439041,NaN,NaN,NaN
2014-07-17,96.478966,96.464819,96.464819,96.478966,96.477551,68154.646540,6.468773,5.893226,4.998864,NaN,...,100.0,0.644448,NaN,NaN,96736.370743,1.219569,91.301258,NaN,NaN,NaN
2014-07-18,102.700314,102.695411,102.695411,102.700314,102.699823,111480.682755,6.448398,6.468773,5.893226,4.998864,...,100.0,1.215521,NaN,NaN,208217.053497,0.273810,95.951569,NaN,NaN,NaN


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['volume'] = stock_data[stock_symbols[i]]['volume'] / stock_data[stock_symbols[i]]['volume'].ewm(5).mean()

stock_data[stock_symbols[9]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,rsi,macd,roc,atr,obv,emv,mfi,cmf,stoch,cci
Symbols,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB,...,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB,SAB
date,,,,,,,,,,,,,,,,,,,,,
2016-12-06,132.000000,132.000000,132.000000,132.000000,132.000000,1.000000,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
2016-12-07,138.814815,138.814815,138.814815,138.814815,138.814815,0.444413,5.162738,NaN,NaN,NaN,...,100.0,0.118822,NaN,NaN,802.592593,0.000000,21.899900,NaN,NaN,NaN
2016-12-08,147.089983,147.089983,147.089983,147.089983,147.089983,0.640641,5.961300,5.162738,NaN,NaN,...,100.0,0.450786,NaN,NaN,1739.247941,0.000000,38.543338,NaN,NaN,NaN
2016-12-09,156.599192,149.670214,149.670214,156.599192,151.993071,1.329704,6.464892,5.961300,5.162738,NaN,...,100.0,1.065604,NaN,NaN,4044.108949,18171.910973,60.352065,NaN,NaN,NaN
2016-12-12,167.185317,164.783955,164.783955,167.185317,165.588983,3.025815,6.760013,6.464892,5.961300,5.162738,...,100.0,2.010623,NaN,NaN,28163.886333,1279.337983,92.077820,NaN,NaN,NaN


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]].replace(0, np.nan, inplace=True)
  stock_data[stock_symbols[i]] = stock_data[stock_symbols[i]].dropna()

stock_data[stock_symbols[8]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,rsi,macd,roc,atr,obv,emv,mfi,cmf,stoch,cci
Symbols,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,...,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN,MSN
date,,,,,,,,,,,,,,,,,,,,,
2014-01-23,95.614629,92.005241,93.787128,93.982277,93.676399,0.728467,0.035034,-2.785228,-0.270295,4.187274,...,83.824630,2.432133,8.897518,2.371062,2.180240e+06,-21.007217,84.468307,0.246258,71.506429,92.854166
2014-01-24,96.515120,93.301834,94.250495,95.943797,94.796740,0.564197,2.087117,0.035034,-2.785228,-0.270295,...,85.209709,2.730516,8.556775,2.478142,2.363514e+06,19.260374,84.851555,0.276771,82.743265,94.677554
2014-01-27,96.180292,93.430642,95.387673,95.330329,94.714359,0.537937,-0.639403,2.087117,0.035034,-2.785228,...,82.319316,2.986299,2.496427,2.579393,2.204189e+06,-1.777759,81.094623,0.297210,79.228930,77.878944
2014-02-06,94.763102,92.500725,93.835686,93.165615,93.457026,0.935312,-2.270750,-0.639403,2.087117,0.035034,...,72.568987,3.158443,-3.856790,2.691144,1.930844e+06,-9.713081,75.270955,0.240015,66.188985,49.716835
2014-02-07,93.617086,91.200254,92.642490,91.432965,91.781959,1.106961,-1.859752,-2.270750,-0.639403,2.087117,...,65.764895,3.232405,-5.389088,2.758190,1.600029e+06,-8.936648,69.162044,0.149014,54.919555,23.725978


In [ ]:
for i in range(len(stock_symbols)):
  stock_data[stock_symbols[i]]['trend', str(stock_symbols[i])] = (stock_data[stock_symbols[i]]['today'].iloc[:] > 0).astype(int)

In [ ]:
stock_data[stock_symbols[7]].head()

Attributes,high,low,open,close,avg,volume,today,previous1,previous2,previous3,...,macd,roc,atr,obv,emv,mfi,cmf,stoch,cci,trend
Symbols,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,...,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG,HPG
date,,,,,,,,,,,,,,,,,,,,,
2014-01-23,50.385817,48.269779,49.581593,48.655930,49.100764,1.094812,-5.264565,-3.015248,3.798205,3.704092,...,1.857512,7.099856,1.538113,5.914569e+06,-6.094757,80.117165,0.320337,61.667579,75.408468,0
2014-01-24,48.315036,46.794423,48.033557,47.189575,47.423267,1.182011,-3.013722,-5.264565,-3.015248,3.798205,...,1.991270,0.884884,1.623207,4.989444e+06,-2.914366,71.285116,0.219815,49.856273,34.691766,0
2014-01-27,47.070263,45.693048,46.776745,46.156351,46.348644,0.992205,-2.189518,-3.013722,-5.264565,-3.015248,...,2.041625,-6.179586,1.680860,4.214141e+06,-2.083798,64.854457,0.193824,39.889702,5.358501,0
2014-02-06,48.129592,45.307567,45.686861,47.614723,46.830525,0.899301,3.159634,-2.189518,-3.013722,-5.264565,...,2.057995,-6.672154,1.807523,4.902446e+06,1.381376,66.140456,0.220017,49.596967,15.217021,1
2014-02-07,48.695357,46.667648,47.840401,47.670153,47.363184,0.981502,0.116414,3.159634,-2.189518,-3.013722,...,2.048816,-9.982555,1.892601,5.650808e+06,2.609070,67.083788,0.175218,49.814677,24.577911,1


In [ ]:
df = {}
for i in range(len(stock_symbols)):
  df[stock_symbols[i]] = stock_data[stock_symbols[i]][['today', 'previous1', 'previous2', 'previous3', 'previous4', 'previous5', 'volume',
                                                  'ema50', 'ema21', 'ema14', 'ema5', 'rsi', 'macd', 'roc', 'atr', 'obv', 'cmf', 'emv', 'stoch', 'mfi', 'cci', 'trend']]

df[stock_symbols[6]].head()

Attributes,today,previous1,previous2,previous3,previous4,previous5,volume,ema50,ema21,ema14,...,macd,roc,atr,obv,cmf,emv,stoch,mfi,cci,trend
Symbols,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,...,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT,FPT
date,,,,,,,,,,,,,,,,,,,,,
2014-01-23,2.046120,0.577065,1.676399,1.209760,1.664779,1.069379,0.745576,1.081930,1.076486,1.071957,...,0.931884,8.524583,1.152735,8.547579e+06,-0.042632,1.828867,96.061665,100.000000,142.530604,1
2014-01-24,2.510089,2.046120,0.577065,1.676399,1.209760,1.664779,0.998033,1.100850,1.094088,1.088475,...,1.112992,10.071564,1.259585,9.284341e+06,-0.015861,4.130713,95.114123,100.000000,156.981061,1
2014-01-27,0.269010,2.510089,2.046120,0.577065,1.676399,1.209760,0.907201,1.096368,1.088566,1.082131,...,1.300499,8.560377,1.342805,9.941282e+06,-0.004475,2.134234,91.323246,100.000000,145.406973,1
2014-02-06,-1.078962,0.269010,2.510089,2.046120,0.577065,1.676399,0.829337,1.078836,1.070451,1.063615,...,1.469531,6.105431,1.411559,9.361295e+06,-0.005302,-1.782669,84.279961,94.143762,110.878741,0
2014-02-07,-1.567412,-1.078962,0.269010,2.510089,2.046120,0.577065,0.992582,1.057998,1.049445,1.042584,...,1.600400,2.720319,1.476449,8.668206e+06,-0.059782,-0.833797,74.485446,87.767647,83.860608,0


#Xây dựng mô hình LSTM

##VIC

In [ ]:
X = df[stock_symbols[0]].loc[:, df[stock_symbols[0]].columns != ('trend', 'VIC')]
y = df[stock_symbols[0]].loc[:, df[stock_symbols[0]].columns == ('trend', 'VIC')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (1932, 30, 21)
y train dataset shape (1932, 1)
X test dataset shape (491, 30, 21)
y test dataset shape (491, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 128)           76800     
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=16)

Epoch 1/15
121/121 [==============================] - 10s 46ms/step - loss: 0.2525
Epoch 2/15
121/121 [==============================] - 6s 48ms/step - loss: 0.2502
Epoch 3/15
121/121 [==============================] - 5s 40ms/step - loss: 0.2503
Epoch 4/15
121/121 [==============================] - 7s 54ms/step - loss: 0.2492
Epoch 5/15
121/121 [==============================] - 5s 39ms/step - loss: 0.2485
Epoch 6/15
121/121 [==============================] - 5s 39ms/step - loss: 0.2448
Epoch 7/15
121/121 [==============================] - 6s 53ms/step - loss: 0.2405
Epoch 8/15
121/121 [==============================] - 5s 39ms/step - loss: 0.2350
Epoch 9/15
121/121 [==============================] - 6s 48ms/step - loss: 0.2328
Epoch 10/15
121/121 [==============================] - 5s 45ms/step - loss: 0.2307
Epoch 11/15
121/121 [==============================] - 5s 40ms/step - loss: 0.2291
Epoch 12/15
121/121 [==============================] - 6s 54ms/step - loss: 0.2279
Epoch 13/15


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)

62/62 [==============================] - 1s 11ms/step


In [ ]:
prediction = (y_pred > 0.5).astype(int)

In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy:  0.6354378818737271
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.65      0.69       307
           1       0.51      0.61      0.56       184

    accuracy                           0.64       491
   macro avg       0.62      0.63      0.62       491
weighted avg       0.65      0.64      0.64       491



##VCB

In [ ]:
X = df[stock_symbols[1]].loc[:, df[stock_symbols[1]].columns != ('trend', 'VCB')]
y = df[stock_symbols[1]].loc[:, df[stock_symbols[1]].columns == ('trend', 'VCB')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (1950, 30, 21)
y train dataset shape (1950, 1)
X test dataset shape (495, 30, 21)
y test dataset shape (495, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=16)

Epoch 1/15
122/122 [==============================] - 14s 43ms/step - loss: 0.2517
Epoch 2/15
122/122 [==============================] - 6s 50ms/step - loss: 0.2502
Epoch 3/15
122/122 [==============================] - 5s 40ms/step - loss: 0.2500
Epoch 4/15
122/122 [==============================] - 7s 55ms/step - loss: 0.2494
Epoch 5/15
122/122 [==============================] - 5s 40ms/step - loss: 0.2495
Epoch 6/15
122/122 [==============================] - 5s 40ms/step - loss: 0.2489
Epoch 7/15
122/122 [==============================] - 7s 53ms/step - loss: 0.2473
Epoch 8/15
122/122 [==============================] - 5s 39ms/step - loss: 0.2428
Epoch 9/15
122/122 [==============================] - 6s 52ms/step - loss: 0.2364
Epoch 10/15
122/122 [==============================] - 5s 42ms/step - loss: 0.2335
Epoch 11/15
122/122 [==============================] - 5s 40ms/step - loss: 0.2334
Epoch 12/15
122/122 [==============================] - 7s 54ms/step - loss: 0.2309
Epoch 13/15


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 1s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6181818181818182
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.61      0.64       273
           1       0.57      0.63      0.60       222

    accuracy                           0.62       495
   macro avg       0.62      0.62      0.62       495
weighted avg       0.62      0.62      0.62       495



##VNM

In [ ]:
X = df[stock_symbols[2]].loc[:, df[stock_symbols[2]].columns != ('trend', 'VNM')]
y = df[stock_symbols[2]].loc[:, df[stock_symbols[2]].columns == ('trend', 'VNM')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)


X train dataset shape (1946, 30, 21)
y train dataset shape (1946, 1)
X test dataset shape (495, 30, 21)
y test dataset shape (495, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_21 (LSTM)              (None, 64)                49408     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=30, batch_size=16)

Epoch 1/30
122/122 [==============================] - 11s 55ms/step - loss: 0.2520
Epoch 2/30
122/122 [==============================] - 7s 61ms/step - loss: 0.2499
Epoch 3/30
122/122 [==============================] - 7s 55ms/step - loss: 0.2493
Epoch 4/30
122/122 [==============================] - 5s 40ms/step - loss: 0.2495
Epoch 5/30
122/122 [==============================] - 6s 46ms/step - loss: 0.2491
Epoch 6/30
122/122 [==============================] - 6s 47ms/step - loss: 0.2484
Epoch 7/30
122/122 [==============================] - 5s 40ms/step - loss: 0.2455
Epoch 8/30
122/122 [==============================] - 7s 53ms/step - loss: 0.2376
Epoch 9/30
122/122 [==============================] - 5s 40ms/step - loss: 0.2305
Epoch 10/30
122/122 [==============================] - 5s 42ms/step - loss: 0.2299
Epoch 11/30
122/122 [==============================] - 6s 51ms/step - loss: 0.2280
Epoch 12/30
122/122 [==============================] - 5s 40ms/step - loss: 0.2277
Epoch 13/30


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 2s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6242424242424243
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66       274
           1       0.58      0.58      0.58       221

    accuracy                           0.62       495
   macro avg       0.62      0.62      0.62       495
weighted avg       0.62      0.62      0.62       495



##GAS

In [ ]:
X = df[stock_symbols[3]].loc[:, df[stock_symbols[3]].columns != ('trend', 'GAS')]
y = df[stock_symbols[3]].loc[:, df[stock_symbols[3]].columns == ('trend', 'GAS')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1946, 30, 21)
y train dataset shape (1946, 1)
X test dataset shape (495, 30, 21)
y test dataset shape (495, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_33 (LSTM)              (None, 64)                49408     
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
122/122 [==============================] - 20s 63ms/step - loss: 0.2513
Epoch 2/10
122/122 [==============================] - 6s 49ms/step - loss: 0.2505
Epoch 3/10
122/122 [==============================] - 8s 63ms/step - loss: 0.2495
Epoch 4/10
122/122 [==============================] - 6s 48ms/step - loss: 0.2492
Epoch 5/10
122/122 [==============================] - 7s 59ms/step - loss: 0.2477
Epoch 6/10
122/122 [==============================] - 6s 52ms/step - loss: 0.2456
Epoch 7/10
122/122 [==============================] - 6s 53ms/step - loss: 0.2378
Epoch 8/10
122/122 [==============================] - 7s 54ms/step - loss: 0.2327
Epoch 9/10
122/122 [==============================] - 6s 47ms/step - loss: 0.2313
Epoch 10/10
122/122 [==============================] - 7s 58ms/step - loss: 0.2294


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 2s 13ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.597979797979798
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.64      0.60       236
           1       0.63      0.56      0.59       259

    accuracy                           0.60       495
   macro avg       0.60      0.60      0.60       495
weighted avg       0.60      0.60      0.60       495



##HVN

In [ ]:
X = df[stock_symbols[4]].loc[:, df[stock_symbols[4]].columns != ('trend', 'HVN')]
y = df[stock_symbols[4]].loc[:, df[stock_symbols[4]].columns == ('trend', 'HVN')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1351, 30, 21)
y train dataset shape (1351, 1)
X test dataset shape (346, 30, 21)
y test dataset shape (346, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_40 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_41 (LSTM)              (None, 64)                49408     
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=8)

Epoch 1/50
169/169 [==============================] - 12s 46ms/step - loss: 0.2509
Epoch 2/50
169/169 [==============================] - 6s 37ms/step - loss: 0.2487
Epoch 3/50
169/169 [==============================] - 9s 52ms/step - loss: 0.2481
Epoch 4/50
169/169 [==============================] - 7s 42ms/step - loss: 0.2465
Epoch 5/50
169/169 [==============================] - 8s 48ms/step - loss: 0.2440
Epoch 6/50
169/169 [==============================] - 6s 37ms/step - loss: 0.2443
Epoch 7/50
169/169 [==============================] - 8s 47ms/step - loss: 0.2420
Epoch 8/50
169/169 [==============================] - 6s 37ms/step - loss: 0.2391
Epoch 9/50
169/169 [==============================] - 8s 47ms/step - loss: 0.2341
Epoch 10/50
169/169 [==============================] - 6s 38ms/step - loss: 0.2303
Epoch 11/50
169/169 [==============================] - 8s 47ms/step - loss: 0.2256
Epoch 12/50
169/169 [==============================] - 6s 38ms/step - loss: 0.2220
Epoch 13/50


In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

44/44 [==============================] - 1s 12ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6011560693641619
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.65      0.58       144
           1       0.70      0.56      0.62       202

    accuracy                           0.60       346
   macro avg       0.61      0.61      0.60       346
weighted avg       0.62      0.60      0.60       346



##MWG

In [ ]:
X = df[stock_symbols[5]].loc[:, df[stock_symbols[5]].columns != ('trend', 'MWG')]
y = df[stock_symbols[5]].loc[:, df[stock_symbols[5]].columns == ('trend', 'MWG')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1850, 30, 21)
y train dataset shape (1850, 1)
X test dataset shape (471, 30, 21)
y test dataset shape (471, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_42 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_43 (LSTM)              (None, 64)                49408     
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
245/245 [==============================] - 16s 42ms/step - loss: 0.2513
Epoch 2/15
245/245 [==============================] - 12s 47ms/step - loss: 0.2489
Epoch 3/15
245/245 [==============================] - 11s 46ms/step - loss: 0.2491
Epoch 4/15
245/245 [==============================] - 11s 45ms/step - loss: 0.2490
Epoch 5/15
245/245 [==============================] - 10s 40ms/step - loss: 0.2482
Epoch 6/15
245/245 [==============================] - 12s 49ms/step - loss: 0.2446
Epoch 7/15
245/245 [==============================] - 11s 45ms/step - loss: 0.2364
Epoch 8/15
245/245 [==============================] - 12s 48ms/step - loss: 0.2311
Epoch 9/15
245/245 [==============================] - 11s 44ms/step - loss: 0.2302
Epoch 10/15
245/245 [==============================] - 9s 38ms/step - loss: 0.2285
Epoch 11/15
245/245 [==============================] - 11s 45ms/step - loss: 0.2276
Epoch 12/15
245/245 [==============================] - 11s 44ms/step - loss: 0.2280
Ep

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 2s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6068548387096774
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.60      0.53       184
           1       0.72      0.61      0.66       312

    accuracy                           0.61       496
   macro avg       0.60      0.61      0.60       496
weighted avg       0.63      0.61      0.61       496



##FPT

In [ ]:
X = df[stock_symbols[6]].loc[:, df[stock_symbols[6]].columns != ('trend', 'FPT')]
y = df[stock_symbols[6]].loc[:, df[stock_symbols[6]].columns == ('trend', 'FPT')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1954, 30, 21)
y train dataset shape (1954, 1)
X test dataset shape (496, 30, 21)
y test dataset shape (496, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_44 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_45 (LSTM)              (None, 64)                49408     
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
245/245 [==============================] - 18s 45ms/step - loss: 0.2509
Epoch 2/15
245/245 [==============================] - 12s 50ms/step - loss: 0.2499
Epoch 3/15
245/245 [==============================] - 12s 50ms/step - loss: 0.2497
Epoch 4/15
245/245 [==============================] - 12s 49ms/step - loss: 0.2489
Epoch 5/15
245/245 [==============================] - 12s 49ms/step - loss: 0.2484
Epoch 6/15
245/245 [==============================] - 11s 47ms/step - loss: 0.2472
Epoch 7/15
245/245 [==============================] - 11s 46ms/step - loss: 0.2388
Epoch 8/15
245/245 [==============================] - 12s 50ms/step - loss: 0.2327
Epoch 9/15
245/245 [==============================] - 12s 49ms/step - loss: 0.2291
Epoch 10/15
245/245 [==============================] - 13s 54ms/step - loss: 0.2284
Epoch 11/15
245/245 [==============================] - 12s 51ms/step - loss: 0.2280
Epoch 12/15
245/245 [==============================] - 12s 48ms/step - loss: 0.2266
E

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 2s 13ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6048387096774194
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.60      0.54       191
           1       0.71      0.61      0.65       305

    accuracy                           0.60       496
   macro avg       0.60      0.60      0.60       496
weighted avg       0.62      0.60      0.61       496



##HPG

In [ ]:
X = df[stock_symbols[7]].loc[:, df[stock_symbols[7]].columns != ('trend', 'HPG')]
y = df[stock_symbols[7]].loc[:, df[stock_symbols[7]].columns == ('trend', 'HPG')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1952, 30, 21)
y train dataset shape (1952, 1)
X test dataset shape (496, 30, 21)
y test dataset shape (496, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_46 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_47 (LSTM)              (None, 64)                49408     
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
244/244 [==============================] - 14s 40ms/step - loss: 0.2505
Epoch 2/15
244/244 [==============================] - 10s 42ms/step - loss: 0.2492
Epoch 3/15
244/244 [==============================] - 9s 37ms/step - loss: 0.2493
Epoch 4/15
244/244 [==============================] - 10s 39ms/step - loss: 0.2477
Epoch 5/15
244/244 [==============================] - 10s 41ms/step - loss: 0.2422
Epoch 6/15
244/244 [==============================] - 9s 35ms/step - loss: 0.2318
Epoch 7/15
244/244 [==============================] - 10s 41ms/step - loss: 0.2267
Epoch 8/15
244/244 [==============================] - 10s 40ms/step - loss: 0.2238
Epoch 9/15
244/244 [==============================] - 9s 35ms/step - loss: 0.2242
Epoch 10/15
244/244 [==============================] - 10s 42ms/step - loss: 0.2215
Epoch 11/15
244/244 [==============================] - 10s 40ms/step - loss: 0.2214
Epoch 12/15
244/244 [==============================] - 8s 34ms/step - loss: 0.2200
Epoch

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

63/63 [==============================] - 1s 9ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6124497991967871
Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.67      0.57       189
           1       0.74      0.58      0.65       309

    accuracy                           0.61       498
   macro avg       0.62      0.62      0.61       498
weighted avg       0.65      0.61      0.62       498



##MSN

In [ ]:
X = df[stock_symbols[8]].loc[:, df[stock_symbols[8]].columns != ('trend', 'MSN')]
y = df[stock_symbols[8]].loc[:, df[stock_symbols[8]].columns == ('trend', 'MSN')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1946, 30, 21)
y train dataset shape (1946, 1)
X test dataset shape (494, 30, 21)
y test dataset shape (494, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_48 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_49 (LSTM)              (None, 64)                49408     
                                                                 
 dense_24 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=30, batch_size=8)

Epoch 1/30
244/244 [==============================] - 17s 43ms/step - loss: 0.2515
Epoch 2/30
244/244 [==============================] - 10s 42ms/step - loss: 0.2494
Epoch 3/30
244/244 [==============================] - 9s 36ms/step - loss: 0.2496
Epoch 4/30
244/244 [==============================] - 10s 42ms/step - loss: 0.2484
Epoch 5/30
244/244 [==============================] - 10s 43ms/step - loss: 0.2455
Epoch 6/30
244/244 [==============================] - 9s 35ms/step - loss: 0.2387
Epoch 7/30
244/244 [==============================] - 12s 50ms/step - loss: 0.2325
Epoch 8/30
244/244 [==============================] - 10s 42ms/step - loss: 0.2307
Epoch 9/30
244/244 [==============================] - 10s 39ms/step - loss: 0.2279
Epoch 10/30
244/244 [==============================] - 9s 38ms/step - loss: 0.2260
Epoch 11/30
244/244 [==============================] - 10s 42ms/step - loss: 0.2262
Epoch 12/30
244/244 [==============================] - 10s 42ms/step - loss: 0.2246
Epoc

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

62/62 [==============================] - 1s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6194331983805668
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.64      0.63       250
           1       0.62      0.60      0.61       244

    accuracy                           0.62       494
   macro avg       0.62      0.62      0.62       494
weighted avg       0.62      0.62      0.62       494



##SAB

In [ ]:
X = df[stock_symbols[9]].loc[:, df[stock_symbols[9]].columns != ('trend', 'SAB')]
y = df[stock_symbols[9]].loc[:, df[stock_symbols[9]].columns == ('trend', 'SAB')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 30
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []

for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])


X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (1343, 30, 21)
y train dataset shape (1343, 1)
X test dataset shape (344, 30, 21)
y test dataset shape (344, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_50 (LSTM)              (None, 30, 128)           76800     
                                                                 
 lstm_51 (LSTM)              (None, 64)                49408     
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 126273 (493.25 KB)
Trainable params: 126273 (493.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=4)

Epoch 1/15
336/336 [==============================] - 26s 54ms/step - loss: 0.2502
Epoch 2/15
336/336 [==============================] - 14s 42ms/step - loss: 0.2483
Epoch 3/15
336/336 [==============================] - 12s 36ms/step - loss: 0.2460
Epoch 4/15
336/336 [==============================] - 13s 39ms/step - loss: 0.2407
Epoch 5/15
336/336 [==============================] - 13s 38ms/step - loss: 0.2356
Epoch 6/15
336/336 [==============================] - 13s 39ms/step - loss: 0.2274
Epoch 7/15
336/336 [==============================] - 13s 38ms/step - loss: 0.2251
Epoch 8/15
336/336 [==============================] - 13s 39ms/step - loss: 0.2247
Epoch 9/15
336/336 [==============================] - 13s 39ms/step - loss: 0.2211
Epoch 10/15
336/336 [==============================] - 14s 40ms/step - loss: 0.2208
Epoch 11/15
336/336 [==============================] - 13s 39ms/step - loss: 0.2174
Epoch 12/15
336/336 [==============================] - 13s 37ms/step - loss: 0.2172
E

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=8)
prediction = (y_pred > 0.5).astype(int)

43/43 [==============================] - 1s 11ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.6395348837209303
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.61      0.73       270
           1       0.34      0.74      0.47        74

    accuracy                           0.64       344
   macro avg       0.62      0.68      0.60       344
weighted avg       0.78      0.64      0.67       344

